In [10]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import connections
from elasticsearch_dsl import Search
from elasticsearch_dsl import Q

import numpy as np
import pandas as pd
import json

#create new connection to test instance, given an alias 'flagonTest' for later reference
#note: connections are easy enough such that examples are enough, no need for further abstraction
flagonClient = connections.create_connection('flagonTest', hosts=['localhost'], timeout=60)

#TODO describe connections

#hello world test
print(flagonClient)

<Elasticsearch([{'host': 'localhost'}])>


In [2]:
#create new search object, using cluster alias and index name 
#TODO abstract to Query Builder
AleS = Search(using='flagonTest', index="userale") \
    .scan()
    
#execute search
response = AleS.execute()

#print Hello World test for total hits, should match kibana output
print(response.hits.total)


21


In [3]:
#add query in search parameters, match on "click" events
customSearch = AleS.query("match", type="click")

response = customSearch.execute()

print(response.hits.total)

3


In [11]:
print(customSearch.to_dict())

{'query': {'bool': {'filter': [{'term': {'target': 'a.ui big blue button'}}], 'must': [{'match': {'type': 'click'}}]}}}


In [5]:
print(response.to_dict())

{'took': 16, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': 3, 'max_score': 1.8382794, 'hits': [{'_index': 'userale', '_type': 'doc', '_id': 'kJYOV20B7YNHv2dPpmwB', '_score': 1.8382794, '_source': {'path': ['a.ui big red button', 'div.ui center aligned text container', 'div.ui very padded vertical segment', 'div.main-wrapper', 'body', 'html', '#document', 'Window'], 'userId': None, 'useraleVersion': 'demo', 'location': {'x': 462, 'y': 688}, 'toolName': None, 'toolVersion': None, 'clientTime': 1569123638986, 'details': {'alt': False, 'shift': False, 'meta': False, 'clicks': 1, 'ctrl': False}, '@timestamp': '2019-09-22T03:40:39.613Z', 'tags': ['_grokparsefailure'], 'type': 'click', 'host': '172.18.0.1', 'target': 'a.ui big red button', 'userAction': True}}, {'_index': 'userale', '_type': 'doc', '_id': 'mJYPV20B7YNHv2dPIGw7', '_score': 1.8382794, '_source': {'path': ['a.ui big blue button', 'div.ui center aligned text container',

In [12]:
#add filter to search params
customSearch2 = AleS.query("match", type="click") \
    .filter("term", target="a.ui big blue button") \
    .scan()

response = customSearch.execute()

print(response.hits.total)

2


In [13]:
print(customSearch.to_dict())

{'query': {'bool': {'filter': [{'term': {'target': 'a.ui big blue button'}}], 'must': [{'match': {'type': 'click'}}]}}}


In [14]:
print(response.to_dict())

{'took': 10, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': 2, 'max_score': 1.8382794, 'hits': [{'_index': 'userale', '_type': 'doc', '_id': 'mJYPV20B7YNHv2dPIGw7', '_score': 1.8382794, '_source': {'path': ['a.ui big blue button', 'div.ui center aligned text container', 'div.ui very padded vertical segment', 'div.main-wrapper', 'body', 'html', '#document', 'Window'], 'userId': None, 'useraleVersion': 'demo', 'location': {'x': 657, 'y': 705}, 'toolName': None, 'toolVersion': None, 'clientTime': 1569123669151, 'details': {'alt': False, 'shift': False, 'meta': False, 'clicks': 1, 'ctrl': False}, '@timestamp': '2019-09-22T03:41:11.890Z', 'tags': ['_grokparsefailure'], 'type': 'click', 'host': '172.18.0.1', 'target': 'a.ui big blue button', 'userAction': True}}, {'_index': 'userale', '_type': 'doc', '_id': 'pJYPV20B7YNHv2dPJWyZ', '_score': 1.8382794, '_source': {'path': ['a.ui big blue button', 'div.ui center aligned text container

In [17]:
#convert responses to dataframe 
#TODO extract _source(s) from df 
AleData = response.to_dict()
AleMeta = pd.DataFrame.from_dict(AleData["hits"]["hits"])

AleMeta

,_id,_index,_score,_source,_type
0,mJYPV20B7YNHv2dPIGw7,userale,1.838279,"{'path': ['a.ui big blue button', 'div.ui cent...",doc
1,pJYPV20B7YNHv2dPJWyZ,userale,1.838279,"{'path': ['a.ui big blue button', 'div.ui cent...",doc


In [57]:
#TO DO Make Iterator over source dump into python dataframe
AleLogs = json.dumps(AleData["hits"]["hits"][0]['_source']).replace('null', '""')
context = defaults.copy()
context.update(user)
json.loads(AleLogs)

{'path': ['a.ui big blue button',
  'div.ui center aligned text container',
  'div.ui very padded vertical segment',
  'div.main-wrapper',
  'body',
  'html',
  '#document',
  'Window'],
 'userId': '',
 'useraleVersion': 'demo',
 'location': {'x': 657, 'y': 705},
 'toolName': '',
 'toolVersion': '',
 'clientTime': 1569123669151,
 'details': {'alt': False,
  'shift': False,
  'meta': False,
  'clicks': 1,
  'ctrl': False},
 '@timestamp': '2019-09-22T03:41:11.890Z',
 'tags': ['_grokparsefailure'],
 'type': 'click',
 'host': '172.18.0.1',
 'target': 'a.ui big blue button',
 'userAction': True}

In [50]:
#Pull data into pandas series
AleSeries = pd.read_json(AleLogs, orient='index', typ='series')
print(AleSeries)

path              [a.ui big blue button, div.ui center aligned t...
userId                                                             
useraleVersion                                                 demo
location                                       {'x': 657, 'y': 705}
toolName                                                           
toolVersion                                                        
clientTime                                            1569123669151
details           {'alt': False, 'shift': False, 'meta': False, ...
@timestamp                                 2019-09-22T03:41:11.890Z
tags                                            [_grokparsefailure]
type                                                          click
host                                                     172.18.0.1
target                                         a.ui big blue button
userAction                                                     True
dtype: object


In [51]:
#Retrieve field
AleSeries['path']

['a.ui big blue button',
 'div.ui center aligned text container',
 'div.ui very padded vertical segment',
 'div.main-wrapper',
 'body',
 'html',
 '#document',
 'Window']